# 지하철 1호선 역간 소요시간 분석
## 인접한 역 사이의 이동 시간 계산 및 분석

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3
import numpy as np
from datetime import datetime, timedelta

DB_PATH = r"..\data\input\metro_datetime.db"
TABLE = "Metro_Line_1_Schedule"

## 1. 전체 데이터 로드

In [3]:
# 데이터 로드
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
        SELECT 
            `역사코드`,
            `역사명`,
            `주중주말`,
            `방향`,
            `급행여부`,
            `열차코드`,
            `열차도착시간_dt`,
            `열차출발시간_dt`,
            `출발역`,
            `도착역`
        FROM {TABLE}
        WHERE `급행여부` = 0
        ORDER BY `열차코드`, `열차도착시간_dt`
    """
    df = pd.read_sql(query, conn)

print(f"총 데이터 수: {len(df):,}행")
print(f"고유 열차 수: {df['열차코드'].nunique():,}대")
print(f"주중주말 구분: {df['주중주말'].unique()}")

# datetime 변환
df['열차도착시간_dt'] = pd.to_datetime(df['열차도착시간_dt'])
df['열차출발시간_dt'] = pd.to_datetime(df['열차출발시간_dt'])

display(df.head(10))

총 데이터 수: 68,791행
고유 열차 수: 708대
주중주말 구분: ['DAY' 'END' 'SAT']


,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간_dt,열차출발시간_dt,출발역,도착역
0,1701,구로,DAY,DOWN,0,K1,NaT,2025-07-04 05:00:00,구로,인천
1,1701,구로,END,DOWN,0,K1,NaT,2025-07-04 05:00:00,구로,인천
2,1701,구로,SAT,DOWN,0,K1,NaT,2025-07-04 05:00:00,구로,인천
3,1813,구일,DAY,DOWN,0,K1,2025-07-04 05:02:00,2025-07-04 05:02:30,구로,인천
4,1813,구일,END,DOWN,0,K1,2025-07-04 05:02:00,2025-07-04 05:02:30,구로,인천
5,1813,구일,SAT,DOWN,0,K1,2025-07-04 05:02:00,2025-07-04 05:02:30,구로,인천
6,1801,개봉,DAY,DOWN,0,K1,2025-07-04 05:04:00,2025-07-04 05:04:30,구로,인천
7,1801,개봉,END,DOWN,0,K1,2025-07-04 05:04:00,2025-07-04 05:04:30,구로,인천
8,1801,개봉,SAT,DOWN,0,K1,2025-07-04 05:04:00,2025-07-04 05:04:30,구로,인천
9,1802,오류동,DAY,DOWN,0,K1,2025-07-04 05:06:30,2025-07-04 05:07:00,구로,인천


## 2. 역간 소요시간 계산

각 열차의 연속된 정차역 사이의 이동 시간을 계산합니다.
- 현재 역의 출발시간 - 다음 역의 도착시간 = 역간 소요시간

In [4]:
def calculate_inter_station_time(df):
    """
    역간 소요시간을 계산하는 함수
    
    Parameters:
    -----------
    df : DataFrame
        열차 스케줄 데이터
    
    Returns:
    --------
    DataFrame
        역간 소요시간 정보
    """
    # 결과를 저장할 리스트
    travel_times = []
    
    # 열차코드와 주중주말별로 그룹화
    for (train_code, day_type), group in df.groupby(['열차코드', '주중주말']):
        # 도착시간 기준으로 정렬
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        # 연속된 역들 사이의 소요시간 계산
        for i in range(len(group) - 1):
            current_station = group.iloc[i]
            next_station = group.iloc[i + 1]
            
            # 현재 역 출발시간과 다음 역 도착시간이 있는 경우만 계산
            if pd.notna(current_station['열차출발시간_dt']) and pd.notna(next_station['열차도착시간_dt']):
                # 소요시간 계산 (분 단위)
                travel_time = (next_station['열차도착시간_dt'] - current_station['열차출발시간_dt']).total_seconds() / 60
                
                # 음수이거나 너무 큰 값 제외 (데이터 오류)
                if 0 < travel_time < 60:  # 1시간 이내의 정상적인 값만
                    travel_times.append({
                        '출발역명': current_station['역사명'],
                        '도착역명': next_station['역사명'],
                        '출발역코드': current_station['역사코드'],
                        '도착역코드': next_station['역사코드'],
                        '소요시간_분': travel_time,
                        '열차코드': train_code,
                        '주중주말': day_type,
                        '방향': current_station['방향'],
                        '출발시간': current_station['열차출발시간_dt'],
                        '도착시간': next_station['열차도착시간_dt']
                    })
    
    return pd.DataFrame(travel_times)

# 역간 소요시간 계산
print("⏱️ 역간 소요시간 계산 중...")
df_travel_time = calculate_inter_station_time(df)

print(f"\n✅ 총 {len(df_travel_time):,}개의 역간 이동 기록 생성")
print(f"고유 역 구간 수: {df_travel_time.groupby(['출발역명', '도착역명']).ngroups:,}개")

display(df_travel_time.head(20))

⏱️ 역간 소요시간 계산 중...

✅ 총 65,526개의 역간 이동 기록 생성
고유 역 구간 수: 199개


,출발역명,도착역명,출발역코드,도착역코드,소요시간_분,열차코드,주중주말,방향,출발시간,도착시간
0,구일,개봉,1813,1801,1.5,K1,DAY,DOWN,2025-07-04 05:02:30,2025-07-04 05:04:00
1,개봉,오류동,1801,1802,2.0,K1,DAY,DOWN,2025-07-04 05:04:30,2025-07-04 05:06:30
2,오류동,온수,1802,1821,2.5,K1,DAY,DOWN,2025-07-04 05:07:00,2025-07-04 05:09:30
3,온수,역곡,1821,1803,1.5,K1,DAY,DOWN,2025-07-04 05:10:00,2025-07-04 05:11:30
4,역곡,소사,1803,1814,2.0,K1,DAY,DOWN,2025-07-04 05:12:00,2025-07-04 05:14:00
5,소사,부천,1814,1804,1.5,K1,DAY,DOWN,2025-07-04 05:14:30,2025-07-04 05:16:00
6,부천,중동,1804,1822,2.0,K1,DAY,DOWN,2025-07-04 05:16:30,2025-07-04 05:18:30
7,중동,송내,1822,1805,1.5,K1,DAY,DOWN,2025-07-04 05:19:00,2025-07-04 05:20:30
8,송내,부개,1805,1815,1.5,K1,DAY,DOWN,2025-07-04 05:21:00,2025-07-04 05:22:30
9,부개,부평,1815,1806,2.0,K1,DAY,DOWN,2025-07-04 05:23:00,2025-07-04 05:25:00


## 3. 역 구간 정규화

"서울역→남영" 과 "남영→서울역" 을 같은 구간으로 처리합니다.

In [5]:
def normalize_station_pair(row):
    """
    역 쌍을 정규화하는 함수
    알파벳순으로 정렬하여 항상 같은 형태로 만듦
    """
    stations = sorted([row['출발역명'], row['도착역명']])
    return f"{stations[0]} ↔ {stations[1]}"

# 정규화된 구간명 추가
df_travel_time['역구간'] = df_travel_time.apply(normalize_station_pair, axis=1)

print("✅ 역 구간 정규화 완료")
print(f"정규화된 고유 구간 수: {df_travel_time['역구간'].nunique():,}개\n")

# 예시 출력
print("예시: 정규화된 구간")
display(df_travel_time[['출발역명', '도착역명', '역구간', '소요시간_분']].head(10))

✅ 역 구간 정규화 완료
정규화된 고유 구간 수: 102개

예시: 정규화된 구간


,출발역명,도착역명,역구간,소요시간_분
0,구일,개봉,개봉 ↔ 구일,1.5
1,개봉,오류동,개봉 ↔ 오류동,2.0
2,오류동,온수,오류동 ↔ 온수,2.5
3,온수,역곡,역곡 ↔ 온수,1.5
4,역곡,소사,소사 ↔ 역곡,2.0
5,소사,부천,부천 ↔ 소사,1.5
6,부천,중동,부천 ↔ 중동,2.0
7,중동,송내,송내 ↔ 중동,1.5
8,송내,부개,부개 ↔ 송내,1.5
9,부개,부평,부개 ↔ 부평,2.0


## 4. 역간 평균 소요시간 계산

같은 구간을 운행하는 모든 열차의 평균 소요시간을 계산합니다.

In [7]:
# 역 구간별 통계 계산
station_stats = df_travel_time.groupby('역구간').agg({
    '소요시간_분': ['mean', 'std', 'min', 'max', 'count']
}).round(2)

# 컬럼명 단순화
station_stats.columns = ['평균_분', '표준편차', '최소_분', '최대_분', '측정횟수']
station_stats = station_stats.reset_index()

# 소요시간이 긴 순서로 정렬
station_stats_sorted = station_stats.sort_values('평균_분', ascending=False)

print("⏱️ 역간 평균 소요시간 Top 30 (소요시간이 긴 구간):")
print("=" * 100)
display(station_stats_sorted.head(30))

# CSV로 저장
station_stats_sorted.to_csv('..\data\output\역간_소요시간_전체.csv', index=False, encoding='utf-8-sig')
print("\n✅ '역간_소요시간_전체.csv' 파일로 저장되었습니다.")

⏱️ 역간 평균 소요시간 Top 30 (소요시간이 긴 구간):


<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
C:\Users\JBK\AppData\Local\Temp\ipykernel_8352\73170402.py:18: SyntaxWarning: invalid escape sequence '\d'
  station_stats_sorted.to_csv('..\data\output\역간_소요시간_전체.csv', index=False, encoding='utf-8-sig')


,역구간,평균_분,표준편차,최소_분,최대_분,측정횟수
90,연천 ↔ 전곡,7.50,0.00,7.5,7.5,59
73,성환 ↔ 평택,6.50,0.04,6.0,6.5,360
27,당정 ↔ 성균관대,5.50,0.00,5.5,5.5,2
57,병점 ↔ 서동탄,5.00,0.00,5.0,5.0,136
77,소요산 ↔ 청산,5.00,0.50,4.5,5.5,118
10,광명 ↔ 금천구청,5.00,0.00,5.0,5.0,60
32,덕정 ↔ 지행,4.50,0.03,4.0,4.5,345
31,덕계 ↔ 양주,4.50,0.00,4.5,4.5,345
49,두정 ↔ 직산,4.50,0.03,4.0,4.5,360
86,신창 ↔ 온양온천,4.50,0.00,4.5,4.5,95



✅ '역간_소요시간_전체.csv' 파일로 저장되었습니다.


## 5. 주중/주말별 비교

In [ ]:
# 주중/주말별 평균 소요시간
station_stats_by_day = df_travel_time.groupby(['역구간', '주중주말']).agg({
    '소요시간_분': ['mean', 'count']
}).round(2)

station_stats_by_day.columns = ['평균_분', '측정횟수']
station_stats_by_day = station_stats_by_day.reset_index()

# Pivot으로 주중/주말 비교
comparison = station_stats_by_day.pivot(index='역구간', columns='주중주말', values='평균_분')
comparison['차이(주중-주말)'] = comparison.get('DAY', 0) - comparison.get('SAT', 0)
comparison = comparison.reset_index()

# 전체 평균 소요시간과 결합
comparison = comparison.merge(
    station_stats[['역구간', '평균_분']], 
    on='역구간', 
    how='left'
)
comparison = comparison.sort_values('평균_분', ascending=False)

print("📊 주중/주말 소요시간 비교 Top 20:")
display(comparison.head(20))

# CSV로 저장
comparison.to_csv('..\data\output\역간_소요시간_주중주말비교.csv', index=False, encoding='utf-8-sig')
print("\n✅ '역간_소요시간_주중주말비교.csv' 파일로 저장되었습니다.")

📊 주중/주말 소요시간 비교 Top 20:


<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\JBK\AppData\Local\Temp\ipykernel_16716\879298547.py:26: SyntaxWarning: invalid escape sequence '\d'
  comparison.to_csv('..\data\output\역간_소요시간_주중주말비교.csv', index=False, encoding='utf-8-sig')


,역구간,DAY,END,SAT,차이(주중-주말),평균_분
90,연천 ↔ 전곡,7.50,7.50,7.50,0.00,7.50
73,성환 ↔ 평택,6.49,6.50,6.50,-0.01,6.50
27,당정 ↔ 성균관대,NaN,5.50,5.50,NaN,5.50
57,병점 ↔ 서동탄,5.00,5.00,5.00,0.00,5.00
77,소요산 ↔ 청산,5.00,5.00,5.00,0.00,5.00
10,광명 ↔ 금천구청,5.00,5.00,5.00,0.00,5.00
32,덕정 ↔ 지행,4.50,4.50,4.50,0.00,4.50
31,덕계 ↔ 양주,4.50,4.50,4.50,0.00,4.50
49,두정 ↔ 직산,4.50,4.50,4.50,0.00,4.50
86,신창 ↔ 온양온천,4.50,4.50,4.50,0.00,4.50



✅ '역간_소요시간_주중주말비교.csv' 파일로 저장되었습니다.


## 6. 시각화: 소요시간이 긴 구간 Top 30

In [ ]:
# Top 30 구간 막대 그래프
top30 = station_stats_sorted.head(30).copy()

fig = px.bar(
    top30,
    x='평균_분',
    y='역구간',
    orientation='h',
    title='역간 평균 소요시간 Top 30 (소요시간이 긴 구간)',
    labels={'평균_분': '평균 소요시간 (분)', '역구간': '역 구간'},
    color='평균_분',
    color_continuous_scale='Reds',
    text='평균_분'
)

fig.update_traces(texttemplate='%{text:.1f}분', textposition='outside')
fig.update_layout(
    height=900,
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)

fig.show()

## 7. 시각화: 소요시간 분포

In [ ]:
# 전체 소요시간 분포
fig = px.histogram(
    df_travel_time,
    x='소요시간_분',
    nbins=50,
    title='역간 소요시간 분포 (전체)',
    labels={'소요시간_분': '소요시간 (분)', 'count': '빈도'},
    color_discrete_sequence=['steelblue']
)

fig.add_vline(
    x=df_travel_time['소요시간_분'].mean(), 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"평균: {df_travel_time['소요시간_분'].mean():.1f}분"
)

fig.show()

print(f"\n📊 소요시간 통계:")
print(f"평균: {df_travel_time['소요시간_분'].mean():.2f}분")
print(f"중앙값: {df_travel_time['소요시간_분'].median():.2f}분")
print(f"표준편차: {df_travel_time['소요시간_분'].std():.2f}분")
print(f"최소: {df_travel_time['소요시간_분'].min():.2f}분")
print(f"최대: {df_travel_time['소요시간_분'].max():.2f}분")


📊 소요시간 통계:
평균: 2.14분
중앙값: 2.00분
표준편차: 0.84분
최소: 1.00분
최대: 7.50분


## 8. 특정 구간 상세 분석

가장 시간이 오래 걸리는 구간을 상세히 분석합니다.

In [ ]:
# 가장 오래 걸리는 구간
longest_section = station_stats_sorted.iloc[0]['역구간']

print(f"🔍 가장 오래 걸리는 구간: {longest_section}")
print("=" * 100)

# 해당 구간의 모든 측정값 조회
section_detail = df_travel_time[df_travel_time['역구간'] == longest_section].copy()
section_detail = section_detail.sort_values('소요시간_분', ascending=False)

print(f"\n총 측정 횟수: {len(section_detail)}회")
print(f"평균 소요시간: {section_detail['소요시간_분'].mean():.2f}분")
print(f"최소 소요시간: {section_detail['소요시간_분'].min():.2f}분")
print(f"최대 소요시간: {section_detail['소요시간_분'].max():.2f}분")

print("\n주중/주말별 평균:")
display(section_detail.groupby('주중주말')['소요시간_분'].agg(['mean', 'count']).round(2))

print("\n방향별 평균:")
display(section_detail.groupby('방향')['소요시간_분'].agg(['mean', 'count']).round(2))

# 시각화
fig = px.box(
    section_detail,
    x='주중주말',
    y='소요시간_분',
    color='방향',
    title=f'구간 "{longest_section}" 소요시간 분포',
    labels={'소요시간_분': '소요시간 (분)', '주중주말': '요일 구분'},
    points='all'
)
fig.show()

🔍 가장 오래 걸리는 구간: 연천 ↔ 전곡

총 측정 횟수: 59회
평균 소요시간: 7.50분
최소 소요시간: 7.50분
최대 소요시간: 7.50분

주중/주말별 평균:


,mean,count
주중주말,,
DAY,7.5,21
END,7.5,19
SAT,7.5,19



방향별 평균:


,mean,count
방향,,
UP,7.5,59


## 9. 소요시간 변동성이 큰 구간

표준편차가 큰 구간 = 운행 시간이 불안정한 구간

In [ ]:
# 표준편차가 큰 구간 (측정 횟수 10회 이상만)
high_variance = station_stats[station_stats['측정횟수'] >= 10].copy()
high_variance = high_variance.sort_values('표준편차', ascending=False)

print("⚠️ 소요시간 변동성이 큰 구간 Top 20:")
print("(표준편차가 크다 = 운행 시간이 불안정)")
print("=" * 100)
display(high_variance.head(20))

# 시각화
fig = px.scatter(
    station_stats,
    x='평균_분',
    y='표준편차',
    size='측정횟수',
    hover_data=['역구간'],
    title='역간 소요시간: 평균 vs 변동성',
    labels={'평균_분': '평균 소요시간 (분)', '표준편차': '표준편차 (분)'},
    color='표준편차',
    color_continuous_scale='Reds'
)
fig.show()

⚠️ 소요시간 변동성이 큰 구간 Top 20:
(표준편차가 크다 = 운행 시간이 불안정)


,역구간,평균_분,표준편차,최소_분,최대_분,측정횟수
96,전곡 ↔ 청산,3.75,0.75,3.0,4.5,118
77,소요산 ↔ 청산,5.00,0.50,4.5,5.5,118
43,동두천 ↔ 소요산,3.39,0.21,3.0,3.5,173
12,광운대 ↔ 월계,1.54,0.13,1.5,2.0,653
50,두정 ↔ 천안,4.00,0.06,4.0,5.0,279
30,덕계 ↔ 덕정,3.00,0.05,3.0,3.5,345
93,오산 ↔ 진위,3.00,0.05,3.0,3.5,360
73,성환 ↔ 평택,6.50,0.04,6.0,6.5,360
101,평택 ↔ 평택지제,4.00,0.04,3.5,4.5,360
92,오산 ↔ 오산대,3.50,0.04,3.0,3.5,360


## 10. 소요시간이 짧은 구간 (빠른 구간)

In [ ]:
# 소요시간이 짧은 구간 Top 20
shortest_sections = station_stats_sorted.tail(20).copy()
shortest_sections = shortest_sections.sort_values('평균_분')

print("⚡ 소요시간이 짧은 구간 Top 20 (빠른 구간):")
print("=" * 100)
display(shortest_sections)

# 시각화
fig = px.bar(
    shortest_sections,
    x='평균_분',
    y='역구간',
    orientation='h',
    title='역간 평균 소요시간 Top 20 (빠른 구간)',
    labels={'평균_분': '평균 소요시간 (분)', '역구간': '역 구간'},
    color='평균_분',
    color_continuous_scale='Greens',
    text='평균_분'
)

fig.update_traces(texttemplate='%{text:.1f}분', textposition='outside')
fig.update_layout(
    height=600,
    yaxis={'categoryorder': 'total descending'}
)

fig.show()

⚡ 소요시간이 짧은 구간 Top 20 (빠른 구간):


,역구간,평균_분,표준편차,최소_분,최대_분,측정횟수
29,대방 ↔ 신길,1.0,0.00,1.0,1.0,1250
85,신이문 ↔ 외대앞,1.0,0.00,1.0,1.0,1084
26,녹천 ↔ 창동,1.5,0.00,1.5,1.5,687
33,도봉 ↔ 도봉산,1.5,0.00,1.5,1.5,688
97,제기동 ↔ 청량리,1.5,0.00,1.5,1.5,1145
64,부천 ↔ 소사,1.5,0.00,1.5,1.5,658
6,개봉 ↔ 구일,1.5,0.00,1.5,1.5,658
16,군포 ↔ 당정,1.5,0.02,1.5,2.0,644
38,도화 ↔ 제물포,1.5,0.00,1.5,1.5,648
39,도화 ↔ 주안,1.5,0.00,1.5,1.5,648


## 11. 양방향 소요시간 비교 검증

서울역→남영과 남영→서울역의 소요시간이 정말 같은지 검증합니다.

In [ ]:
# 특정 구간의 양방향 소요시간 비교
def compare_bidirectional(df, station1, station2):
    """
    두 역 사이의 양방향 소요시간을 비교
    """
    # 방향1: station1 → station2
    dir1 = df[
        (df['출발역명'] == station1) & 
        (df['도착역명'] == station2)
    ]['소요시간_분']
    
    # 방향2: station2 → station1
    dir2 = df[
        (df['출발역명'] == station2) & 
        (df['도착역명'] == station1)
    ]['소요시간_분']
    
    if len(dir1) > 0 and len(dir2) > 0:
        print(f"\n📍 {station1} ↔ {station2} 구간:")
        print(f"  {station1} → {station2}: 평균 {dir1.mean():.2f}분 (측정 {len(dir1)}회)")
        print(f"  {station2} → {station1}: 평균 {dir2.mean():.2f}분 (측정 {len(dir2)}회)")
        print(f"  차이: {abs(dir1.mean() - dir2.mean()):.2f}분")
        
        return {
            '구간': f"{station1} ↔ {station2}",
            f'{station1}→{station2}': round(dir1.mean(), 2),
            f'{station2}→{station1}': round(dir2.mean(), 2),
            '차이': round(abs(dir1.mean() - dir2.mean()), 2)
        }
    return None

# 여러 구간 비교
test_pairs = [
    ('서울역', '남영'),
    ('남영', '용산'),
    ('용산', '노량진'),
    ('서울역', '시청'),
    ('시청', '종각'),
    ('종각', '종로3가'),
]

print("🔍 양방향 소요시간 비교 검증")
print("=" * 100)

comparison_results = []
for station1, station2 in test_pairs:
    result = compare_bidirectional(df_travel_time, station1, station2)
    if result:
        comparison_results.append(result)

if comparison_results:
    print("\n\n📊 양방향 비교 요약:")
    df_comparison = pd.DataFrame(comparison_results)
    display(df_comparison)

🔍 양방향 소요시간 비교 검증

📍 서울역 ↔ 남영 구간:
  서울역 → 남영: 평균 2.50분 (측정 620회)
  남영 → 서울역: 평균 2.50분 (측정 627회)
  차이: 0.00분

📍 남영 ↔ 용산 구간:
  남영 → 용산: 평균 2.00분 (측정 623회)
  용산 → 남영: 평균 2.00분 (측정 627회)
  차이: 0.00분

📍 용산 ↔ 노량진 구간:
  용산 → 노량진: 평균 3.00분 (측정 623회)
  노량진 → 용산: 평균 3.00분 (측정 627회)
  차이: 0.00분

📍 서울역 ↔ 시청 구간:
  서울역 → 시청: 평균 2.00분 (측정 625회)
  시청 → 서울역: 평균 2.00분 (측정 626회)
  차이: 0.00분

📍 시청 ↔ 종각 구간:
  시청 → 종각: 평균 2.00분 (측정 638회)
  종각 → 시청: 평균 2.00분 (측정 626회)
  차이: 0.00분

📍 종각 ↔ 종로3가 구간:
  종각 → 종로3가: 평균 1.50분 (측정 638회)
  종로3가 → 종각: 평균 1.50분 (측정 626회)
  차이: 0.00분


📊 양방향 비교 요약:


,구간,서울역→남영,남영→서울역,차이,남영→용산,용산→남영,용산→노량진,노량진→용산,서울역→시청,시청→서울역,시청→종각,종각→시청,종각→종로3가,종로3가→종각
0,서울역 ↔ 남영,2.5,2.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,남영 ↔ 용산,NaN,NaN,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,용산 ↔ 노량진,NaN,NaN,0.0,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,서울역 ↔ 시청,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
4,시청 ↔ 종각,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
5,종각 ↔ 종로3가,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,1.5


## 12. 전체 노선의 소요시간 히트맵

In [ ]:
# 역 순서대로 정렬하기 위해 역사코드 사용
# 출발역과 도착역의 평균 코드 계산
section_with_code = df_travel_time.copy()
section_with_code['평균코드'] = (section_with_code['출발역코드'] + section_with_code['도착역코드']) / 2

# 구간별 평균 소요시간과 평균 코드
heatmap_data = section_with_code.groupby('역구간').agg({
    '소요시간_분': 'mean',
    '평균코드': 'mean'
}).reset_index()

# 코드 순으로 정렬
heatmap_data = heatmap_data.sort_values('평균코드')

# Top 50 구간만 표시 (가독성)
heatmap_top50 = heatmap_data.nlargest(50, '소요시간_분')

fig = px.bar(
    heatmap_top50,
    y='역구간',
    x='소요시간_분',
    orientation='h',
    title='소요시간 Top 50 구간 (역 순서 고려)',
    labels={'소요시간_분': '평균 소요시간 (분)', '역구간': '역 구간'},
    color='소요시간_분',
    color_continuous_scale='RdYlGn_r'
)

fig.update_layout(height=1200)
fig.show()

## 💡 분석 결론

### 주요 발견사항:

1. **소요시간이 가장 긴 구간**: 
   - 역간 거리가 멀거나 표정속도가 낮은 구간
   - 교외 구간에서 주로 발견됨

2. **양방향 소요시간 일치 확인**:
   - 상행/하행의 평균 소요시간이 거의 동일함
   - 정규화를 통한 통합 분석이 타당함

3. **소요시간 변동성**:
   - 일부 구간은 표준편차가 큼 → 운행 시간 불안정
   - 혼잡도, 신호 대기 등의 영향 추정

### 혼잡도 분석과의 연계:

- **소요시간이 긴 구간**: 승객 대기 시간 증가 → 혼잡도 누적
- **배차 간격과 결합**: 소요시간 × 배차 간격 = 승객 경험 품질
- **변동성 큰 구간**: 운행 불확실성 → 승객 불편 증가

### 다음 분석 방향:

1. 소요시간 × 배차 간격 = 복합 지표 생성
2. 시간대별 소요시간 변화 분석
3. 차량 노후도와 소요시간 상관관계
4. 실제 혼잡도 데이터와 결합하여 원인 규명